In [1]:
import pandas as pd

df = pd.read_csv('spokesperson.csv')
df.head()

,Unnamed: 0,link,date,text
0,0,https://www.fmprc.gov.cn/mfa_eng/xwfw_665399/s...,2021-12-22,China Daily: As we are getting closer to the o...
1,1,https://www.fmprc.gov.cn/mfa_eng/xwfw_665399/s...,2021-12-22,Zhao Lijian: The idea of a green Beijing 2022 ...
2,2,https://www.fmprc.gov.cn/mfa_eng/xwfw_665399/s...,2021-12-22,"In recent years, Beijing and Hebei province ha..."
3,3,https://www.fmprc.gov.cn/mfa_eng/xwfw_665399/s...,2021-12-22,"During the Games, all 12 Olympic venues will b..."
4,4,https://www.fmprc.gov.cn/mfa_eng/xwfw_665399/s...,2021-12-22,"Lately, many people in the international commu..."


In [2]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

In [3]:
pattern = [
    [{"TEXT": "economy"}]]

matcher.add("Economy", pattern)

In [4]:
stat_list=[]

for i in range(len(df)):
    text= df['text'][i]
    stat_list.append(text)

len(stat_list)

20197

In [6]:
import re
from tqdm import tqdm
econ_stats=[]

for i in tqdm(range(len(stat_list))):
    text= nlp(stat_list[i])
    matches= matcher(text)
    for match_id, start, end in matches:
        string_id= nlp.vocab.strings[match_id] 
        span = text[start:end]
        span_sent= re.findall(r'[^.]*' + re.escape(str(span)) + r'[^.]*\.', str(text))
        econ_stats.append([(span_sent),(df['date'][i])])
len(econ_stats)

100%|██████████| 20197/20197 [03:39<00:00, 91.85it/s] 


670

In [7]:
from transformers import BertTokenizer

In [8]:
sent_df = pd.DataFrame(econ_stats, columns=['statements', 'date'])
#sent_df.to_csv('sent-df')
sent_df= pd.read_csv('sent-df')
sent_df.head()

,Unnamed: 0,statements,date
0,0,[' Without UN Security Council’s authorization...,2021-12-21
1,1,"[' In fact, the US’ oppression of Chinese ente...",2021-12-20
2,2,"[' Apart from humanitarian assistance, China h...",2021-12-20
3,3,[' What the US did has seriously undermined th...,2021-12-17
4,4,[' They shared the hope to strengthen cooperat...,2021-12-16


In [9]:
stat_ser = sent_df['statements']

In [10]:
model_name = 'ProsusAI/finbert'
from transformers import BertTokenizer, BertForSequenceClassification

In [11]:
model = BertForSequenceClassification.from_pretrained(model_name)

In [12]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [13]:
import torch.nn.functional as F
import torch

preds_list=[]

for i in tqdm(range(len(stat_ser))):
    tokens = tokenizer.encode_plus(str(stat_ser[i]), max_length=512,
                                   truncation=True, padding='max_length',
                                   return_tensors='pt')
    output = model(**tokens)
    probs= F.softmax(output[0], dim=-1)
    pred= torch.argmax(probs)
    preds_list.append(pred)

100%|██████████| 670/670 [17:31<00:00,  1.57s/it]


In [14]:
sent_df['sentiment'] = preds_list

In [19]:
sent_df.head()

,Unnamed: 0,statements,date,sentiment
0,0,[' Without UN Security Council’s authorization...,2021-12-21,tensor(1)
1,1,"[' In fact, the US’ oppression of Chinese ente...",2021-12-20,tensor(1)
2,2,"[' Apart from humanitarian assistance, China h...",2021-12-20,tensor(0)
3,3,[' What the US did has seriously undermined th...,2021-12-17,tensor(1)
4,4,[' They shared the hope to strengthen cooperat...,2021-12-16,tensor(0)


In [54]:
sent_df.statements.iloc[621]

'[]'

In [20]:
sent_df=sent_df.reset_index(drop=True)

In [22]:
print('Number of positive sentences is' + ' ' + str(len(sent_df[sent_df.sentiment==0])))
print('\n')
print('Number of neutral sentences is' + ' ' + str(len(sent_df[sent_df.sentiment==1])))
print('\n')
print('Number of negative sentences is' + ' ' + str(len(sent_df[sent_df.sentiment==2])))



Number of positive sentences is 402


Number of neutral sentences is 101


Number of negative sentences is 167


In [55]:
import numpy as np
from numpy import random
my_rand= random.randint(len(sent_df), size=(10))


for i in my_rand:
    x = sent_df.sentiment[i]
    if x == 0:
        print(sent_df['statements'][i], "\n" + "\n" + 'the sentiment for statement'+ ' '+ str(i) + ' '+ 'is' + ' '+ 'Positive' + "\n" + "------------------" + "\n" )
    elif x ==1:
        print(sent_df['statements'][i], "\n" + "\n" + 'the sentiment for statement'+ ' '+ str(i) + ' '+ 'is' + ' '+ 'Neutral' + "\n" + "------------------" + "\n" )
    elif x==2:
        print(sent_df['statements'][i], "\n" + "\n" + 'the sentiment for statement'+ ' '+ str(i) + ' '+ 'is' + ' '+ 'Negative' + "\n" + "------------------" + "\n" )
    else:
        print('there is no sentiment')

[' It will not only bring about more fruitful data cooperation between the two sides, but will also promote the sound development of digital economy and the building of a community with a shared future in cyberspace by injecting the wisdom and strength of developing countries into global digital governance.'] 

the sentiment for statement 230 is Positive
------------------

[' If our economy is impacted, there will inevitably be spillover effects on other countries and even the world economy.'] 

the sentiment for statement 633 is Neutral
------------------

[' Is this also part of the framework you mentioned? In a world where the global economy is deeply integrated, the US moves to undermine the stability of global industrial and supply chains are against the trend.'] 

the sentiment for statement 17 is Neutral
------------------

[' He pointed out that China will deeply engage in international cooperation on green and low-carbon development and the digital economy, and work actively 